# SOLID design principles

2. Open-closed principle (OCP)
* "software entities should be open for extension, but closed for modification"
* Extensibility: new features can be added without modifying existing code 
* Stability: reduces the risk of introducing bugs when making changes 
* Flexibility: adapts to changing requirements more easily

I also learned from this example:
* enum.Enum class in python
* operator overloading in python (bitwise `and` operator &)
* `map()` function in python

In [1]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

class Product:
    def __init__(self, name, color, size):
        self.name = name 
        self.color = color 
        self.size = size 

In [2]:
# old implememntation: state space explosion 
# 3 criteria
# 7 methods: c s w cs sw cw csw
class ProductFilter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p 

    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p 

    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p 

In [3]:
apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)

products = [apple, tree, house]

In [4]:
pf = ProductFilter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f' - {p.name} is green')

Green products (old):
 - Apple is green
 - Tree is green


In [16]:
# OCP: open for extension, closed for modification
# filter using the specification pattern 
class Specification:
    def is_satisfied(self, item):
        pass 
    # and operator makes life easier 
    def __and__(self, other):
        return AndSpecification(self, other)

class Filter:
    def filter(self, items, sepc):
        pass 

class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color 
    
    def is_satisfied(self, item):
        return item.color == self.color 
    
class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size 
    
    def is_satisfied(self, item):
        return item.size == self.size 
    
class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args 
    
    def is_satisfied(self, item):
        return all(map(
            # ???
            lambda spec: spec.is_satisfied(item), self.args ))
    
class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item 


In [17]:
bf = BetterFilter()

print('Green products (new)')
green = ColorSpecification(color=Color.GREEN)
for p in bf.filter(products, green):
    print(f' - {p.name} is green')

Green products (new)
 - Apple is green
 - Tree is green


In [18]:
print('Large products: ')
large = SizeSpecification(size=Size.LARGE)
for p in bf.filter(products, large):
    print(f' - {p.name} is large')

Large products: 
 - Tree is large
 - House is large


In [19]:
print('Large blue items: ')
# large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))

# checkout the '&' ampersand operator overloading above 
large_blue = large & ColorSpecification(Color.BLUE)

for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue ')

Large blue items: 
 - House is large and blue 
